In [61]:
import numpy as np
import pandas as pd
import math
import scipy.stats as sp
import plotly
from plotly import graph_objs as go
plotly.offline.init_notebook_mode(connected = True)

# Finite Difference Methods

## Explicit Finite Difference Method (a)

In [4]:
def explicit_FD(S,K,T,r,sig,dx=0,q=0,opt_type='c',N=50,Nj=50):
    if opt_type == 'c':
        c = 1 
    else:
        c = -1 
    dt = T/N
    if dx == 0:
        dx = sig*np.sqrt(3*dt)
    nu = r-q-.5*sig**2
    pu = .5*dt*((sig/dx)**2+(nu/dx))
    pm = 1-dt*(sig/dx)**2 - r*dt
    pd = .5*dt*((sig/dx)**2-(nu/dx))
    
    #asset values at maturity -Nj*dx to Nj*dx
    stock = np.zeros(2*Nj+1)
    stock[0] = S*np.exp(-Nj*dx)
    for i in range(1,2*Nj+1):
        stock[i]=stock[i-1]*np.exp(dx)
    
    #option Values at maturity
    option = np.zeros((N+1,2*Nj+1))
    for i in range(0,2*Nj+1):
        option[N,i] = max(0,c*(stock[i]-K))

    #step back to calculate each u(i,j) using u(i+1,j+1), u(i+1,j), u(i+1,j-1)
    for i in range(N-1,-1,-1):
        for j in range(1,2*Nj):
            option[i,j]=pu*option[i+1,j+1]+pm*option[i+1,j]+pd*option[i+1,j-1]
        #Boundary Conditions
        if opt_type == 'c':
            # S = 0
            option[i,0] = option[i,1]
            # S = inf
            option[i,2*Nj] = option[i,2*Nj-1]+(stock[2*Nj]-stock[2*Nj-1])
        else:
            # S = 0
            option[i,0]=option[i,1] + (stock[1]-stock[0])
            # S = inf
            option[i,2*Nj]=option[i,2*Nj-1]

    return option[0,Nj]

#### Explicit Finite Difference European Call

In [213]:
S = 100
K = 100
T = 1
sig = .25
r = .06
q = .03
explicit_FD(S,K,T,r,sig,q=q,opt_type='c',N=100,Nj=100)

10.993214983722249

#### Explicit Finite Difference European Put

In [6]:
explicit_FD(S,K,T,r,sig,q=q,opt_type='p',N=100,Nj=100)

8.123419836666503

## Implicit Finite Difference Method (b)

In [214]:
def solve_imp_tridiag(M,A,B,C,lL,lU,Nj):
    #substitute boundary conditions
    pmp = np.zeros(2*Nj)
    pp = np.zeros(2*Nj)
    pmp[1] = B+C
    pp[1] = M[0,1] + C*lL
    
    #eliminate upper diagonal
    for j in range(2,2*Nj):
        pmp[j] = B - A*C/pmp[j-1]
        pp[j] = M[0,j] - pp[j-1]*C/pmp[j-1]

    #use boundary condition @ j = Nj and equation @ j = Nj-1
    M[1,2*Nj] = (pp[2*Nj-1] + pmp[2*Nj-1]*lU)/(A + pmp[2*Nj-1])
    M[1,2*Nj-1] = M[1,2*Nj] - lU

    #back substitution
    for j in range(2*Nj-2,-1,-1):
        M[1,j] = (pp[j] - A*M[1,j+1])/pmp[j]
    
    M[1,0] = M[1,1]-lL

    return M

#Implicit Finite Difference function
def implicit_FD(S,K,T,r,sig,dx=0,q=0,opt_type='c',N=50,Nj=50):
    if opt_type == 'c':
        c = 1 
    else:
        c = -1
    dt = T/N
    if dx == 0:
        dx = sig*np.sqrt(3*dt)
    nu = r-q-.5*sig**2
    A = -.5*dt*((sig/dx)**2+(nu/dx))
    B = 1+dt*(sig/dx)**2 + r*dt
    C = -.5*dt*((sig/dx)**2-(nu/dx))
    
    #asset values at maturity
    stock = np.zeros(2*Nj+1)
    stock[0] = S*np.exp(-Nj*dx)
    for i in range(1,2*Nj+1):
        stock[i]=stock[i-1]*np.exp(dx)
    
    #option Values at maturity
    option = np.zeros((2,2*N+1))
    for j in range(0,2*Nj+1):
        option[0,j] = max(0,c*(stock[j]-K))
    
    #boundary conditions
    if opt_type == 'c':
        lambdaU = stock[2*Nj] - stock[2*Nj - 1]
        lambdaL = 0
    else:
        lambdaU = 0
        lambdaL = -1*(stock[1]-stock[0])

    #step back by solving the tridiagonal matrix
    for i in range(N-1,-1,-1):
        option = solve_imp_tridiag(option,A,B,C,lambdaL,
                                   lambdaU,Nj)
        option[0,:] = option[1,:]
    return option[0,Nj]

#### Implicit Finite Difference European Call

In [215]:
implicit_FD(S,K,T,r,sig,q=q,opt_type='c',N=100,Nj=100)

/Users/Brendon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning:

divide by zero encountered in double_scalars



10.964014174587897

#### Implicit Finite Difference European Put

In [4]:
def reverse_words_order_and_swap_cases(sentence):
    ans = ""
    j = 0
    temp = ""
    for i in range(len(sentence)):
        if sentence[i] == " ":
            ans = temp + ans
            temp = ""
        elif sentence[i].islower():
            temp = temp + sentence[i].upper()
        else:
            temp = temp + sentence[i].lower()
        return temp + ans



0
1
2
3
4


In [9]:
explicit_FD(S,K,T,r,sig,q=q,opt_type='p',N=100,Nj=100)

8.123419836666503

## Crank-Nicolson Finite Difference Method (c)

In [39]:
def solve_imp_CN(M,A,B,C,lL,lU,Nj):
    #substitute boundary conditions
    pmp = np.zeros(2*Nj)
    pp = np.zeros(2*Nj)
    pmp[1] = B+C
    pp[1] = -A*M[0,2]-(B-2)*M[0,1]-C*M[0,0] + C*lL
    
    #eliminate upper diagonal
    for j in range(2,2*Nj):
        pmp[j] = B - A*C/pmp[j-1]
        pp[j] = -A*M[0,j+1]-(B-2)*M[0,j]-C*M[0,j-1]-pp[j-1]*C/pmp[j-1]
    
    #use boundary condition @ j = Nj and equation @ j = Nj-1
    M[1,2*Nj] = (pp[2*Nj-1] + pmp[2*Nj-1]*lU)/(A + pmp[2*Nj-1])
    M[1,2*Nj-1] = M[1,2*Nj] - lU
    
    #back substitution
    for j in range(2*Nj-2,-1,-1):
        M[1,j] = (pp[j] - A*M[1,j+1])/pmp[j]

    M[1,0] = M[1,1]-lL

    return M
    
def cn_FD(S,K,T,r,sig,dx=0,q=0,opt_type='c',N=50,Nj=50):
    if opt_type == 'c':
        c = 1 
    else:
        c = -1
    dt = T/N
    if dx == 0:
        dx = sig*np.sqrt(3*dt)
    nu = r-q-.5*sig**2
    A = -.25*dt*((sig/dx)**2+(nu/dx))
    B = 1+.5*dt*(sig/dx)**2 + .5*r*dt
    C = -.25*dt*((sig/dx)**2-(nu/dx))
    
    #asset values at maturity
    stock = np.zeros(2*Nj+1)
    stock[0] = S*np.exp(-Nj*dx)
    for i in range(1,2*Nj+1):
        stock[i]=stock[i-1]*np.exp(dx)
        
    #option values at maturity
    option = np.zeros((2,2*N+1))
    for j in range(0,2*Nj+1):
        option[0,j] = max(0,c*(stock[j]-K))
    
    #boundary conditions
    if opt_type == 'c':
        lambdaU = stock[2*Nj] - stock[2*Nj -1]
        lambdaL = 0
    else:
        lambdaU = 0
        lambdaL = -1*(stock[1]-stock[0])
    
    #step back thru lattice
    for i in range(N-1,-1,-1):
        option = solve_imp_CN(option,A,B,C,lambdaL,lambdaU,Nj)
        option[0,:] = option[1,:]
            
    return option[0,Nj]

#### Crank-Nicolson Finite Difference European Call

In [11]:
cn_FD(S,K,T,r,sig,q=q,opt_type='c',N=100,Nj=100)

/Users/Brendon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in double_scalars


10.978637958258984

#### Crank-Nicolson Finite Difference European Put

In [12]:
cn_FD(S,K,T,r,sig,q=q,opt_type='p',N=100,Nj=100)

/Users/Brendon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in double_scalars


8.110101844769272

## Estimating Parameters for ε = 0.001 (d)

For both the Explicit and Implicit Finite Difference schemes estimate the numbers ∆t, ∆x, and the total number Nj of points on the space gridxneeded to obtain a desired error of ε= 0.001.

Order of convergence for both schemes is O(dx^2 + dt)

Assuming the error is equal to the order of convergence we have ε = dx^2 + dt = 0.001

Using σ = 25% and T = 1 year from the parameters for the next question and the optimal convergence condition for Explicit of dx = σ*sqrt(3*dt), we have:

ε = (σ*sqrt(3*dt))^2 + dt = 3*dt*σ^2 + dt = dt*(3*σ^2 + 1)

Solving for dt we have:
dt = ε/(3*σ^2 + 1) = 0.001/(3*.25^2 + 1) = 0.000842

In [18]:
# dt
dt = 0.001/(3*.25**2 + 1)
dt

0.0008421052631578948

Solving for dx:
ε = dx^2 + dt => dx = sqrt(ε - dt) = sqrt(0.001 - 0.000842) = 0.012565

In [22]:
# dx
dx = np.sqrt(.001-dt)
dx

0.012565617248750865

Solving for n:
dt = T/n => n = T/dt = 1/0.000842 = 1188

In [36]:
# n
n = math.ceil(T/dt)
n

1188

Taking altnerate formulation of n: 
n = 3*((2*Nj + 1)/nSD)^2 
such that nSD is the number of standard deviations around the mean to consider for the range of the asset value

Solving for Nj:
Nj = (nSD*sqrt(n/3) - 1)/2

Assuming we take nSD = 6 for a range 3 standard deviations on each side of the mean to represent the entire possible range at T:
Nj = (6*sqrt(1188/3) - 1)/2 = 60

In [35]:
# Nj
Nj = math.ceil((6*np.sqrt(1188/3) - 1)/2)
Nj

60

## European Calls and Puts For Each Method Using Calculated dt, dx, n, and Nj (e)

In [99]:
price_table = pd.DataFrame([[explicit_FD(S,K,T,r,sig,dx=dx,q=q,opt_type='c',N=n,Nj=Nj),
  explicit_FD(S,K,T,r,sig,dx=dx,q=q,opt_type='p',N=n,Nj=Nj)],
[implicit_FD(S,K,T,r,sig,dx=dx,q=q,opt_type='c',N=n,Nj=Nj),
implicit_FD(S,K,T,r,sig,dx=dx,q=q,opt_type='p',N=n,Nj=Nj)],
[cn_FD(S,K,T,r,sig,dx=dx,q=q,opt_type='c',N=n,Nj=Nj),
cn_FD(S,K,T,r,sig,dx=dx,q=q,opt_type='p',N=n,Nj=Nj)]], columns=['Call','Put'], index=['EFD','IFD','CNFD']
)
price_table

/Users/Brendon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning:

divide by zero encountered in double_scalars



,Call,Put
EFD,11.011566,8.143203
IFD,11.009137,8.140981
CNFD,11.010352,8.142092


## Empirical Number of Iterations Using Black-Scholes (f)

In [55]:
def BSM(S0, K, tau, r, sigma, opt_type='c', q=0):
    d1 = (np.log(S0/K)+(r-q+.5*sigma**2)*tau)/(sigma*np.sqrt(tau))
    d2 = d1 - sigma*np.sqrt(tau)
    N = lambda x: sp.norm.cdf(x)
    if opt_type == 'c':
        return S0*np.exp(-q*tau)*N(d1) - np.exp(-r*tau)*K*N(d2)
    else:
        return K*np.exp(-r*tau)*N(-d2) - S0*np.exp(-q*tau)*N(-d1)
    
def param_calc(f,opt_type,start,f_name,eps=0.001):
    Nj = start
    n = math.ceil(3*((2*Nj + 1)/6)**2)
    dt = T/n
    dx = sig*np.sqrt(3*dt)
    BS_price = BSM(S,K,T,r,sig,opt_type,q)
    price = f(S,K,T,r,sig,dx,q,opt_type,n,Nj)
    err = abs(price - BS_price)
    while err > eps:
        Nj += 1
        n = math.ceil(3*((2*Nj + 1)/6)**2)
        dt = T/n
        dx = sig*np.sqrt(3*dt)
        price = f(S,K,T,r,sig,dx,q,opt_type,n,Nj)
        err = abs(price - BS_price)
    df = pd.DataFrame([[opt_type,BS_price,price,dx,dt,n,Nj]], index = [f_name], 
                      columns = ['Type', 'BS_Price', 'FD_Price', 'dx', 'dt', 'N', 'Nj'])
    return df

In [60]:
summary_df = pd.DataFrame(columns = ['Type', 'BS_Price', 'FD_Price', 'dx', 'dt', 'N', 'Nj'])
for FD_fun in [explicit_FD,implicit_FD,cn_FD]:
    if FD_fun == explicit_FD:
        f_name = 'EFD'
    elif FD_fun == implicit_FD:
        f_name = 'IFD'
    else:
        f_name = 'CNFD'
    temp = param_calc(FD_fun,'c',10,f_name)
    summary_df = summary_df.append(temp)
    temp = param_calc(FD_fun,'p',10,f_name)
    summary_df = summary_df.append(temp)
    
summary_df

/Users/Brendon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in double_scalars


,Type,BS_Price,FD_Price,dx,dt,N,Nj
EFD,c,11.013079,11.012095,0.010487,0.000587,1705,71
EFD,p,8.144979,8.143999,0.009433,0.000475,2107,79
IFD,c,11.013079,11.012094,0.006666,0.000237,4219,112
IFD,p,8.144979,8.143995,0.006329,0.000214,4681,118
CNFD,c,11.013079,11.012100,0.007936,0.000336,2977,94
CNFD,p,8.144979,8.143988,0.007462,0.000297,3367,100


All methods were able to calculate prices that converged to the Black Scholes price; however the number of steps to convergence varied for each method and in each scenario the put required an extra 6-8 steps for convergence. The explicit method had the quickest convergence followed by the Crank-Nicolson while the implicit was the last too converge requiring the most steps. This is contrary to the widely believed claim that the Crank-Nicolson method should be the most stable of the three methods and the fastest to converge with an order of convergence of 
O(dx^2 + (dt/2)) compared to O(dx^2 + dt) for the others.

## Graph of the Implicit Finite Difference Values pu, pm, pd as a Function of σ (g)

In [81]:
dt = T/n
def f_pu(sig):
    dx = sig*np.sqrt(3*dt)
    nu = r-q-.5*sig**2
    A = -.5*dt*((sig/dx)**2+(nu/dx))
    return A
def f_pm(sig):
    dx = sig*np.sqrt(3*dt)
    nu = r-q-.5*sig**2
    B = 1+dt*(sig/dx)**2 + r*dt
    return B
def f_pd(sig):
    dx = sig*np.sqrt(3*dt)
    nu = r-q-.5*sig**2
    C = -.5*dt*((sig/dx)**2-(nu/dx)) 
    return C

As = []
Bs = []
Cs = []
sigs = np.linspace(.05,.6,12)
for sig in sigs:
    As.append(f_pu(sig))
    Bs.append(f_pm(sig))
    Cs.append(f_pd(sig))
    
As = pd.DataFrame(As,index=sigs,columns=['Pu'])
Bs = pd.DataFrame(Bs,index=sigs,columns=['Pm'])
Cs = pd.DataFrame(Cs,index=sigs,columns=['Pd'])

In [90]:
trace0 = go.Scatter(
    x = As.index, y = As['Pu'], name = 'Pu'
)
trace1 = go.Scatter(
    x = Bs.index, y = Bs['Pm'], name = 'Pm', yaxis = 'y2'
)
trace2 = go.Scatter(
    x = Cs.index, y = Cs['Pd'], name = 'Pd'
)
data = [trace0,trace1,trace2]
layout = go.Layout(
    title = 'Implicit Difference Pu, Pm, and Pd as a Function of Sigma', yaxis = dict(title = 'Pu and Pd Value'), 
    yaxis2=dict(
        title="Pm Value",
        anchor="x",
        overlaying="y",
        side="right"
    ), 
        xaxis = dict(title = 'Sigma'))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

The Pu increases with volatility and Pd decreases while Pm remains constant. This is as expected since option prices have a positive sensitivity to volatility and in this case this is shown by the rising Pu, which is the weight applied to u(i,j+1) when calculating u(i+1,j) for the implicit scheme. These values are not to be confused with the actual probabilities used in the explicit scheme that represent the probabilities of a positive, negative, or no movement, and the plot emphasizes that as Pu and Pd are negative and Pm is greater than 1 for all values of sigma. Instead these values more closely represent constant weights for all i's and j's while the volatility is constant.

## Pricing Table (h)

In [107]:
price_table.append(pd.DataFrame([[BSM(S,K,T,r,sig,'c',q),BSM(S,K,T,r,sig,'p',q)]], 
                                index=['BS'], columns=['Call', 'Put']))

,Call,Put
EFD,11.011566,8.143203
IFD,11.009137,8.140981
CNFD,11.010352,8.142092
BS,11.013079,8.144979


In [109]:
abs(price_table['Call']-BSM(S,K,T,r,sig,'c',q))

EFD     0.001512
IFD     0.003942
CNFD    0.002727
Name: Call, dtype: float64

In [110]:
abs(price_table['Put']-BSM(S,K,T,r,sig,'p',q))

EFD     0.001776
IFD     0.003998
CNFD    0.002887
Name: Put, dtype: float64

Using the parameters from (e) all three methods output the same price to the nearest cent for both options although there are variations for the different methods after that point. This is consistent with the results from part (f) that suggested each method required more steps to converge to within 0.001 of the Black-Scholes price than the values calculated in part (d). Moreover the deviations from the Black-Scholes price calculated above show the same result that the explicit method is fastest to converge followed by the Crank-Nicolson and the implicit method is slowest to converge evidenced by it having the largest error from Black-Scholes.

## Calculating Greeks with Explicit Finite Differencing (i)

#### Delta

In [132]:
def delta(f,S,diff=0.01,opt_type='c',n=1188,Nj=60):
    orig = f(S,K,T,r,sig,0,q,opt_type=opt_type,N=n,Nj=Nj)
    new = f(S+diff,K,T,r,sig,0,q,opt_type=opt_type,N=n,Nj=Nj)
    return (new - orig)/diff
# (explicit_FD(S+.01,K,T,r,sig,dx,q,'c',n,Nj)-explicit_FD(S,K,T,r,sig,dx,q,'c',n,Nj))/.01
delta(explicit_FD,S)

0.5885587357052913

#### Gamma

In [154]:
def gamma(f,diff=1,opt_type='c',n=1188,Nj=60):
    orig = delta(explicit_FD,S-diff,opt_type=opt_type,n=n,Nj=Nj)
    new = delta(explicit_FD,S+diff,opt_type=opt_type,n=n,Nj=Nj)
    return (new - orig)/(2*diff)
# (delta(explicit_FD,S+1)-delta(explicit_FD,S-1))/2
gamma(explicit_FD)

0.009439691583601473

#### Theta

In [145]:
def theta(f,diff=1/365,opt_type='c',n=1188,Nj=60):
    orig = f(S,K,T,r,sig,0,q,opt_type=opt_type,N=n,Nj=Nj)
    new = f(S,K,T-diff,r,sig,0,q,opt_type=opt_type,N=n,Nj=Nj)
    return (new - orig)/diff
theta(explicit_FD)

-5.776790704654271

#### Vega

In [147]:
def vega(f,diff=0.001,opt_type='c',n=1188,Nj=60):
    orig = f(S,K,T,r,sig,0,q,opt_type=opt_type,N=n,Nj=Nj)
    new = f(S,K,T,r,sig+diff,0,q,opt_type=opt_type,N=n,Nj=Nj)
    return (new - orig)/diff
vega(explicit_FD)

37.56394156625298

# Fast Fourier Transform

In [221]:
S = 100
eta = .25
t = 1
sig = .25
r = .06
alpha = 1.5
N = 4096

# Black-Scholes Characteristic Funtion
def BS_CF(S,mu,sig,t,phi):
    u = np.log(S) + (mu - .5*sig**2)*t
    vol = t*sig**2
    return np.exp(complex(0,1)*u*phi - .5*vol*phi**2)

def FFT(N,eta,alpha,r,t,S,sig):
    dk = 2*math.pi/(N*eta)
    beta = dk*N/2
    u_arr = eta*np.linspace(0,N-1,N)
    k_arr = -beta + np.linspace(0,N-1,N)*dk
    
    # applying equations 24 and 6 from the paper
    delt_arr = [1] + (N-1)*[0]
    w_arr = eta/3*(3+(-1)**np.linspace(1,N,N))
    psi = (np.exp(-r*t)*BS_CF(S,r,sig,t,u_arr-(alpha+1)*complex(0,1)))/(alpha**2 + alpha-u_arr**2 +
                                                                        complex(0,1)*(2*alpha + 1)*u_arr)
    s_arr = np.exp(complex(0,1)*beta*u_arr)*psi*w_arr
    
    # calling numpy's fft function
    price_FFT = np.fft.fft(s_arr)
    calls_FFT = (np.exp(-alpha*k_arr)/math.pi)*price_FFT.real
    
    Ks = np.exp(k_arr)
    calls_BS = []
    for k in Ks:
        calls_BS.append(BSM(S,k,T,r,sig))
    
    df = pd.DataFrame({'K':Ks, 'BS':calls_BS, 'FFT':calls_FFT})
    return df

In [222]:
temp = FFT(N,eta,alpha,r,t,S,sig)

In [223]:
temp[(temp['K'] > 50) & (temp['K'] < 150)]

,K,BS,FFT
2686,50.134980,52.791133,55.242491
2687,50.443550,52.501140,54.930039
2688,50.754020,52.209410,54.616057
2689,51.066400,51.915938,54.300536
2690,51.380702,51.620717,53.983468
2691,51.696940,51.323741,53.664845
2692,52.015123,51.025002,53.344658
2693,52.335265,50.724496,53.022900
2694,52.657378,50.422218,52.699564
2695,52.981473,50.118161,52.374643


In [224]:
trace0 = go.Scatter(
    x = temp[(temp['K'] > 40) & (temp['K'] < 160)]['K'], y = temp[(temp['K'] > 40) & (temp['K'] < 160)]['BS'], 
    name = 'Black-Scholes'
)
trace1 = go.Scatter(
    x = temp[(temp['K'] > 40) & (temp['K'] < 160)]['K'], y = temp[(temp['K'] > 40) & (temp['K'] < 160)]['FFT'],
    name = 'FFT'
)
data = [trace0, trace1]
layout = go.Layout(
    title = 'Fast Fourier Transform vs Black-Scholes', yaxis = dict(title = 'Price'),  
        xaxis = dict(title = 'Strike'))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [225]:
trace0 = go.Scatter(
    x = temp[(temp['K'] > 40) & (temp['K'] < 160)]['K'], 
    y = abs(temp[(temp['K'] > 40) & (temp['K'] < 160)]['FFT'] - temp[(temp['K'] > 40) & (temp['K'] < 160)]['BS']),
    name = 'Absolute Error'
)
trace1 = go.Scatter(
    x = temp[(temp['K'] > 40) & (temp['K'] < 160)]['K'], 
    y = abs(temp[(temp['K'] > 40) & (temp['K'] < 160)]['FFT'] - 
            temp[(temp['K'] > 40) & (temp['K'] < 160)]['BS'])/temp[(temp['K'] > 40) & (temp['K'] < 160)]['BS']*100,
    name = 'Percent Error', yaxis = 'y2'
)
data = [trace0,trace1]
layout = go.Layout(
    title = 'Fast Fourier Transform Error', yaxis = dict(title = 'Absolute'), 
    yaxis2=dict(
        title="Percent Error (%)",
        anchor="x",
        overlaying="y",
        side="right"
    ), 
        xaxis = dict(title = 'Strike'))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

#### Mean Absolute Error

In [226]:
np.mean(abs(temp[(temp['K'] > 50) & (temp['K'] < 150)]['FFT'] - temp[(temp['K'] > 50) & (temp['K'] < 150)]['BS']))

1.2069939444192253

#### Mean Percent Error

In [227]:
np.mean(abs(temp[(temp['K'] > 50) & (temp['K'] < 150)]['FFT'] - 
            temp[(temp['K'] > 50) & (temp['K'] < 150)]['BS'])/temp[(temp['K'] > 50) & (temp['K'] < 150)]['BS']*100)

9.192930811734449